In [3]:
import pytorch_lightning as pl
from phantoms.models.retrieval.gnn_retrieval_model import GNNRetrievalModel
from massspecgym.data import RetrievalDataset, MassSpecDataModule
from massspecgym.data.transforms import MolFingerprinter
from massspecgym.featurize import SpectrumFeaturizer
from pytorch_lightning.loggers import TensorBoardLogger
from massspecgym.data.datasets import MSnRetrievalDataset
import yaml

In [4]:
# Optional: Set random seed for reproducibility
pl.seed_everything(42)

# Load configuration
config_path = '/Users/macbook/CODE/PhantoMS/phantoms/models/retrieval/configs/config_retrieval.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

Seed set to 42


In [5]:
# Initialize Featurizer
featurizer = SpectrumFeaturizer(config['featurizer'], mode='torch')

In [6]:
# Initialize Dataset
dataset_msn = MSnRetrievalDataset(
    pth=config['data']['file_mgf'],
    candidates_pth=config['data']['file_json'],
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=config['model']['fp_size']),
    max_allowed_deviation=config['data']['max_allowed_deviation']
)

Total valid indices: 15674
Dataset length: 15674


In [7]:
# Initialize DataModule
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=config['data']['batch_size'],
    split_pth=config['data']['split_file'],
    num_workers=config['data']['num_workers']
)

In [13]:
# Initialize Model
model = GNNRetrievalModel(
    hidden_channels=config['model']['hidden_channels'],
    out_channels=config['model']['fp_size'],
    node_feature_dim=config['model']['node_feature_dim'],
)

In [14]:
# Initialize Logger
logger = TensorBoardLogger(save_dir=config['logs']['dir'], name=config['logs']['name'])


In [15]:
# Initialize Trainer
trainer = pl.Trainer(
    accelerator=config['trainer']['accelerator'],
    devices=config['trainer']['devices'],
    max_epochs=config['trainer']['max_epochs'],
    logger=logger,
    log_every_n_steps=config['trainer']['log_every_n_steps'],
    limit_train_batches=config['trainer']['limit_train_batches'],
    limit_val_batches=config['trainer']['limit_val_batches'],
    limit_test_batches=config['trainer']['limit_test_batches'],
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            monitor=config['trainer']['checkpoint_monitor'],
            save_top_k=config['trainer']['save_top_k'],
            mode=config['trainer']['checkpoint_mode'],
            dirpath=config['trainer']['checkpoint_dir'],
            filename='gnn_retrieval-{epoch:02d}-{val_loss:.2f}'
        ),
        pl.callbacks.LearningRateMonitor(logging_interval='step')
    ]
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [16]:
# Train the model
trainer.fit(model, datamodule=data_module_msn)


  | Name    | Type          | Params | Mode 
--------------------------------------------------
0 | gcn1    | GCNLayer      | 133 K  | train
1 | gcn2    | GCNLayer      | 16.5 K | train
2 | head    | RetrievalHead | 544 K  | train
3 | loss_fn | MSELoss       | 0      | train
--------------------------------------------------
694 K     Trainable params
0         Non-trainable params
694 K     Total params
2.778     Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Train dataset size: 11883
Val dataset size: 1899
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


in collate_fn
Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  6.79it/s]in collate_fn
                                                                           

/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 2/2 [00:15<00:00,  0.13it/s, v_num=2, train_loss=0.367]
Validation: |          | 0/? [00:00<?, ?it/s]in collate_fn

Validation DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 12.83it/s]in collate_fn

Epoch 1: 100%|██████████| 2/2 [00:15<00:00,  0.13it/s, v_num=2, train_loss=0.350, val_loss=0.360]
Validation: |          | 0/? [00:00<?, ?it/s]in collate_fn

Validation DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 12.74it/s]in collate_fn

Epoch 2: 100%|██████████| 2/2 [00:15<00:00,  0.13it/s, v_num=2, train_loss=0.340, val_loss=0.348]
Validation: |          | 0/? [00:00<?, ?it/s]in collate_fn

Validation DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 12.73it/s]in collate_fn

Epoch 3: 100%|██████████| 2/2 [00:14<00:00,  0.14it/s, v_num=2, train_loss=0.328, val_loss=0.336]
Validation: |          | 0/? [00:00<?, ?it/s]in collate_fn

Validation DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 12.74it/s]in collate_fn

Epoch 4: 100%|██████████| 2/2 [00:17<00:00, 

`Trainer.fit` stopped: `max_epochs=7` reached.


Epoch 6: 100%|██████████| 2/2 [00:36<00:00,  0.05it/s, v_num=2, train_loss=0.289, val_loss=0.290]


In [17]:
# Test the model
trainer.test(model, datamodule=data_module_msn)

/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Test dataset size: 1892
Testing: |          | 0/? [00:00<?, ?it/s]in collate_fn
Testing DataLoader 0: 100%|██████████| 2/2 [00:19<00:00,  0.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_hit_rate@1                0.0
    test_hit_rate@20                0.0
     test_hit_rate@5                0.0
        test_loss           0.2808142602443695
       test_mces@1          26.774999618530273
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.2808142602443695,
  'test_hit_rate@1': 0.0,
  'test_hit_rate@5': 0.0,
  'test_hit_rate@20': 0.0,
  'test_mces@1': 26.774999618530273}]